In [1]:
# Function to normalize the signal
def normalize_signal(signal):
    max_val = np.max(np.abs(signal))
    if max_val > 0:
        return signal / max_val
    else:
        return signal

In [ ]:
# Iterate over each entry in filtered_signals_lpf_dict
for combo_name, data in filtered_signals_lpf_dict.items():
    # Retrieve the filtered signals and metadata
    lpf_filtered_signals = data['signal']  # Shape: (num_channels, num_samples)
    center_frequencies = data['center_freq']  # List of center frequencies
    bandwidths = data['bandwidth']  # List of bandwidths (if needed)
    
    # Ensure the signals are NumPy arrays
    lpf_filtered_signals = np.array(lpf_filtered_signals)
    
    # Task 10 & 11: Generate cosine signals and modulate with envelopes
    modulated_signals = []
    fs = 16000  # Ensure fs is defined or retrieved from your data
    for idx, ch_signal in enumerate(lpf_filtered_signals):
        center_freq = center_frequencies[idx]
        t = np.arange(len(ch_signal)) / fs
        # Generate the cosine signal at the center frequency
        cosine_signal = np.cos(2 * np.pi * center_freq * t)
        # Task 11: Amplitude modulate the cosine signal with the envelope
        modulated_signal = ch_signal * cosine_signal
        modulated_signals.append(modulated_signal)
    
    modulated_signals = np.array(modulated_signals)
    
    # Task 12: Sum the modulated signals to produce the output signal
    output_signal = np.sum(modulated_signals, axis=0)
    
    # Normalize the output signal to prevent clipping
    output_signal = normalize_signal(output_signal)
    
    # Task 13: Play and save the output sound
    print(f"\nPlaying audio for combination: {combo_name}")
    display(Audio(output_signal, rate=fs))
    
    # Save the output signal to a WAV file
    output_filename = f'output_{combo_name}.wav'
    sf.write(output_filename, output_signal, fs)
    print(f"Output signal saved to: {output_filename}")

In [ ]:
# Function to design and apply the filter bank, rectification, and low-pass filter
def process_signal_with_kwbank(input_signal, num_channels, lowcut, highcut, fs, overlap, filter_type, distr, order, lpf_cutoff, lpf_order):
    """
    Applies the kwbank filter, full-wave rectification, and low-pass filtering.

    Args:
        input_signal (numpy.ndarray): Input audio signal.
        num_channels (int): Number of channels for the kwbank filter.
        lowcut (float): Low cutoff frequency for the kwbank filter.
        highcut (float): High cutoff frequency for the kwbank filter.
        fs (int): Sampling frequency.
        overlap (float): Overlap percentage for the kwbank filter.
        filter_type (str): Filter type for the kwbank filter ('fir' or 'iir').
        distr (str): Distribution type for frequency bands ('erb', 'linear', etc.).
        order (int): Order of the kwbank filter.
        lpf_cutoff (float): Cutoff frequency for the low-pass filter.
        lpf_order (int): Order of the low-pass filter.

    Returns:
        numpy.ndarray: The processed and combined signal.
    """
    # Design the kwbank filter
    kwbank = design_kw_fbank(num_channels, lowcut, highcut, distr, overlap, fs, filter_type, order)
    
    # Apply the bandpass filter bank
    filtered_signals = apply_filter(input_signal, kwbank)
    
    # Full-wave rectification
    rectified_signals = np.abs(np.array(filtered_signals))
    
    # Design the low-pass filter
    nyquist = fs / 2
    normalized_cutoff = lpf_cutoff / nyquist
    lpf_taps = firwin(lpf_order + 1, normalized_cutoff)
    
    # Apply the low-pass filter to each channel
    lpf_filtered_signals = []
    for ch_signal in rectified_signals:
        lpf_filtered_signal = lfilter(lpf_taps, [1.0], ch_signal)
        lpf_filtered_signals.append(lpf_filtered_signal)
    
    # Combine all channels by summing them
    combined_signal = np.sum(lpf_filtered_signals, axis=0)
    
    # Normalize the combined signal
    output_signal = normalize_signal(combined_signal)
    
    return output_signal

# Function to perform a parameter sweep
def parameter_sweep_kwbank(input_signal, fs, param_ranges):
    """
    Perform a parameter sweep for the kwbank filter.

    Args:
        input_signal (numpy.ndarray): Input audio signal.
        fs (int): Sampling frequency.
        param_ranges (dict): Ranges for parameters to sweep over.
            Keys: 'num_channels', 'lowcut', 'highcut', 'overlap', 'order', 'lpf_cutoff', 'lpf_order'

    """
    num_channels_range = param_ranges['num_channels']
    overlap_range = param_ranges['overlap']
    order_range = param_ranges['order']
    lpf_cutoff_range = param_ranges['lpf_cutoff']
    lpf_order_range = param_ranges['lpf_order']

    lowcut = param_ranges['lowcut']
    highcut = param_ranges['highcut']
    distr = param_ranges['distr']
    filter_type = param_ranges['filter_type']

    for num_channels in num_channels_range:
        for overlap in overlap_range:
            for order in order_range:
                for lpf_cutoff in lpf_cutoff_range:
                    for lpf_order in lpf_order_range:
                        print(f"Processing with num_channels={num_channels}, overlap={overlap}, order={order}, lpf_cutoff={lpf_cutoff}, lpf_order={lpf_order}")
                        
                        # Process the signal
                        output_signal = process_signal_with_kwbank(
                            input_signal=input_signal,
                            num_channels=num_channels,
                            lowcut=lowcut,
                            highcut=highcut,
                            fs=fs,
                            overlap=overlap,
                            filter_type=filter_type,
                            distr=distr,
                            order=order,
                            lpf_cutoff=lpf_cutoff,
                            lpf_order=lpf_order
                        )

                        # Play the processed signal
                        display(Audio(output_signal, rate=fs))
                        
                        # Save the output signal
                        filename = f"output_nc{num_channels}_ov{int(overlap*100)}_ord{order}_lpf{lpf_cutoff}_lpf_ord{lpf_order}.wav"
                        sf.write(filename, output_signal, fs)
                        print(f"Saved: {filename}")


In [ ]:
# Define parameter ranges for the sweep
param_ranges = {
    'num_channels': [16, 22, 32],  # Optimal starting point and a close value
    'lowcut': 100,  # Fixed value from project requirements
    'highcut': 8000,  # Fixed value from project requirements
    'overlap': [0.1, 0.2],  # Starting with 10% and 20% overlap
    'order': [16, 32, 64],  # Filter orders to test
    'filter_type': 'fir',  # Fixed as FIR for this sweep
    'distr': 'log2',  # Fixed as ERB for this sweep
    'lpf_cutoff': [400],  # Testing 300 Hz, 400 Hz, and 500 Hz for LPF cutoff
    'lpf_order': [2, 4]  # Testing LPF orders 16 and 32
}

# Call the parameter sweep function
parameter_sweep_kwbank(input_signal=sound, fs=16000, param_ranges=param_ranges)

In [ ]:
# Function to design and apply the IIR Butterworth filter bank, rectification, and low-pass filtering
def process_signal_with_bbank(input_signal, num_channels, lowcut, highcut, fs, overlap, distr, order, lpf_cutoff, lpf_order):
    """
    Applies the bbank filter, full-wave rectification, and low-pass filtering.

    Args:
        input_signal (numpy.ndarray): Input audio signal.
        num_channels (int): Number of channels for the bbank filter.
        lowcut (float): Low cutoff frequency for the bbank filter.
        highcut (float): High cutoff frequency for the bbank filter.
        fs (int): Sampling frequency.
        overlap (float): Overlap percentage for the bbank filter.
        distr (str): Distribution type for frequency bands ('erb', 'log2', etc.).
        order (int): Order of the bbank filter.
        lpf_cutoff (float): Cutoff frequency for the low-pass filter.
        lpf_order (int): Order of the low-pass filter.

    Returns:
        numpy.ndarray: The processed and combined signal.
    """
    # Design the IIR Butterworth filter bank
    bbank = design_iir_butter_fbank(num_channels, lowcut, highcut, distr, overlap, fs, order=order)
    
    # Apply the bandpass filter bank
    filtered_signals = apply_filter(input_signal, bbank)
    
    # Full-wave rectification
    rectified_signals = np.abs(np.array(filtered_signals))
    
    # Design the low-pass filter
    nyquist = fs / 2
    normalized_cutoff = lpf_cutoff / nyquist
    lpf_b, lpf_a = ss.butter(lpf_order, normalized_cutoff, btype='low')
    
    # Apply the low-pass filter to each channel
    lpf_filtered_signals = []
    for ch_signal in rectified_signals:
        lpf_filtered_signal = ss.lfilter(lpf_b, lpf_a, ch_signal)
        lpf_filtered_signals.append(lpf_filtered_signal)
    
    # Combine all channels by summing them
    combined_signal = np.sum(lpf_filtered_signals, axis=0)
    
    # Normalize the combined signal
    output_signal = normalize_signal(combined_signal)
    
    return output_signal

# Function to perform a parameter sweep for the IIR Butterworth filter bank
def parameter_sweep_bbank(input_signal, fs, param_ranges):
    """
    Perform a parameter sweep for the bbank filter.

    Args:
        input_signal (numpy.ndarray): Input audio signal.
        fs (int): Sampling frequency.
        param_ranges (dict): Ranges for parameters to sweep over.
            Keys: 'num_channels', 'lowcut', 'highcut', 'overlap', 'order', 'lpf_cutoff', 'lpf_order'
    """
    num_channels_range = param_ranges['num_channels']
    overlap_range = param_ranges['overlap']
    order_range = param_ranges['order']
    lpf_cutoff_range = param_ranges['lpf_cutoff']
    lpf_order_range = param_ranges['lpf_order']

    lowcut = param_ranges['lowcut']
    highcut = param_ranges['highcut']
    distr = param_ranges['distr']

    for num_channels in num_channels_range:
        for overlap in overlap_range:
            for order in order_range:
                for lpf_cutoff in lpf_cutoff_range:
                    for lpf_order in lpf_order_range:
                        print(f"Processing with num_channels={num_channels}, overlap={overlap}, order={order}, lpf_cutoff={lpf_cutoff}, lpf_order={lpf_order}")
                        
                        # Process the signal
                        output_signal = process_signal_with_bbank(
                            input_signal=input_signal,
                            num_channels=num_channels,
                            lowcut=lowcut,
                            highcut=highcut,
                            fs=fs,
                            overlap=overlap,
                            distr=distr,
                            order=order,
                            lpf_cutoff=lpf_cutoff,
                            lpf_order=lpf_order
                        )

                        # Play the processed signal
                        display(Audio(output_signal, rate=fs))
                        
                        # Save the output signal
                        filename = f"bbank_output_nc{num_channels}_ov{int(overlap*100)}_ord{order}_lpf{lpf_cutoff}_lpf_ord{lpf_order}.wav"
                        sf.write(filename, output_signal, fs)
                        print(f"Saved: {filename}")




In [ ]:
# Define parameter ranges for the sweep
param_ranges_bbank = {
    'num_channels': [8, 16],  # Optimal starting point and a close value
    'lowcut': 100,  # Fixed value from project requirements
    'highcut': 8000,  # Fixed value from project requirements
    'overlap': [0.1],  # Starting with 10% and 20% overlap
    'order': [2, 4, 8],  # Filter orders to test for IIR filters
    'distr': 'linear',  # Fixed as log2 for this sweep
    'lpf_cutoff': [400],  # Testing 300 Hz, 400 Hz, and 500 Hz for LPF cutoff
    'lpf_order': [2, 4]  # Testing LPF orders 2 and 4
}

# Call the parameter sweep function
parameter_sweep_bbank(input_signal=sound, fs=16000, param_ranges=param_ranges_bbank)

In [ ]:
# Function to design and apply the Chebyshev filter bank, rectification, and low-pass filtering
def process_signal_with_chebyshev(input_signal, num_channels, lowcut, highcut, fs, overlap, distr, order, lpf_cutoff, lpf_order, cheby_type):
    """
    Applies the Chebyshev filter bank, full-wave rectification, and low-pass filtering.

    Args:
        input_signal (numpy.ndarray): Input audio signal.
        num_channels (int): Number of channels for the Chebyshev filter bank.
        lowcut (float): Low cutoff frequency for the Chebyshev filter bank.
        highcut (float): High cutoff frequency for the Chebyshev filter bank.
        fs (int): Sampling frequency.
        overlap (float): Overlap percentage for the Chebyshev filter bank.
        distr (str): Distribution type for frequency bands ('log10', etc.).
        order (int): Order of the Chebyshev filter bank.
        lpf_cutoff (float): Cutoff frequency for the low-pass filter.
        lpf_order (int): Order of the low-pass filter.
        cheby_type (str): Type of Chebyshev filter ('cheby1' or 'cheby2').

    Returns:
        numpy.ndarray: The processed and combined signal.
    """
    # Design the Chebyshev filter bank
    chebyshev_bank = design_chebyshev_fbank(num_channels, lowcut, highcut, distr, overlap, fs, filter_type=cheby_type, order=order)
    
    # Apply the bandpass filter bank
    filtered_signals = apply_filter(input_signal, chebyshev_bank)
    
    # Full-wave rectification
    rectified_signals = np.abs(np.array(filtered_signals))
    
    # Design the low-pass filter
    nyquist = fs / 2
    normalized_cutoff = lpf_cutoff / nyquist
    lpf_b, lpf_a = cheby1(lpf_order, 0.1, normalized_cutoff, btype='low') if cheby_type == 'cheby1' else cheby2(lpf_order, 40, normalized_cutoff, btype='low')
    
    # Apply the low-pass filter to each channel
    lpf_filtered_signals = []
    for ch_signal in rectified_signals:
        lpf_filtered_signal = lfilter(lpf_b, lpf_a, ch_signal)
        lpf_filtered_signals.append(lpf_filtered_signal)
    
    # Combine all channels by summing them
    combined_signal = np.sum(lpf_filtered_signals, axis=0)
    
    # Normalize the combined signal
    output_signal = normalize_signal(combined_signal)
    
    return output_signal

# Function to perform a parameter sweep for the Chebyshev filter bank
def parameter_sweep_chebyshev(input_signal, fs, param_ranges, cheby_type):
    """
    Perform a parameter sweep for the Chebyshev filter bank.

    Args:
        input_signal (numpy.ndarray): Input audio signal.
        fs (int): Sampling frequency.
        param_ranges (dict): Ranges for parameters to sweep over.
            Keys: 'num_channels', 'lowcut', 'highcut', 'overlap', 'order', 'lpf_cutoff', 'lpf_order'
        cheby_type (str): Type of Chebyshev filter ('cheby1' or 'cheby2').
    """
    num_channels_range = param_ranges['num_channels']
    overlap_range = param_ranges['overlap']
    order_range = param_ranges['order']
    lpf_cutoff_range = param_ranges['lpf_cutoff']
    lpf_order_range = param_ranges['lpf_order']

    lowcut = param_ranges['lowcut']
    highcut = param_ranges['highcut']
    distr = param_ranges['distr']

    for num_channels in num_channels_range:
        for overlap in overlap_range:
            for order in order_range:
                for lpf_cutoff in lpf_cutoff_range:
                    for lpf_order in lpf_order_range:
                        print(f"Processing with num_channels={num_channels}, overlap={overlap}, order={order}, lpf_cutoff={lpf_cutoff}, lpf_order={lpf_order}, type={cheby_type}")
                        
                        # Process the signal
                        output_signal = process_signal_with_chebyshev(
                            input_signal=input_signal,
                            num_channels=num_channels,
                            lowcut=lowcut,
                            highcut=highcut,
                            fs=fs,
                            overlap=overlap,
                            distr=distr,
                            order=order,
                            lpf_cutoff=lpf_cutoff,
                            lpf_order=lpf_order,
                            cheby_type=cheby_type
                        )

                        # Play the processed signal
                        display(Audio(output_signal, rate=fs))
                        
                        # Save the output signal
                        filename = f"cheby_{cheby_type}_nc{num_channels}_ov{int(overlap*100)}_ord{order}_lpf{lpf_cutoff}_lpf_ord{lpf_order}.wav"
                        sf.write(filename, output_signal, fs)
                        print(f"Saved: {filename}")


In [ ]:
# Define parameter ranges for the sweep
param_ranges_chebyshev = {
    'num_channels': [16, 22],  # Optimal starting point and a close value
    'lowcut': 100,  # Fixed value from project requirements
    'highcut': 8000,  # Fixed value from project requirements
    'overlap': [0.1, 0.2],  # Starting with 10% and 20% overlap
    'order': [4, 8],  # Filter orders to test for Chebyshev filters
    'distr': 'log10',  # Fixed as log10 for this sweep
    'lpf_cutoff': [400],  # Testing 300 Hz, 400 Hz, and 500 Hz for LPF cutoff
    'lpf_order': [2]  # Testing LPF orders 2 and 4
}

# Call the parameter sweep function for Chebyshev Type I
print("Sweeping Chebyshev Type I:")
parameter_sweep_chebyshev(input_signal=sound, fs=16000, param_ranges=param_ranges_chebyshev, cheby_type='cheby1')

In [ ]:
# Define parameter ranges for the sweep
param_ranges_chebyshev = {
    'num_channels': [8],  # Optimal starting point and a close value
    'lowcut': 100,  # Fixed value from project requirements
    'highcut': 8000,  # Fixed value from project requirements
    'overlap': [0.1],  # Starting with 10% and 20% overlap
    'order': [4],  # Filter orders to test for Chebyshev filters
    'distr': 'log10',  # Fixed as log10 for this sweep
    'lpf_cutoff': [400],  # Testing 300 Hz, 400 Hz, and 500 Hz for LPF cutoff
    'lpf_order': [50]  # Testing LPF orders 2 and 4
}

# Call the parameter sweep function for Chebyshev Type II
print("Sweeping Chebyshev Type II:")
parameter_sweep_chebyshev(input_signal=sound, fs=16000, param_ranges=param_ranges_chebyshev, cheby_type='cheby2')

In [ ]:
# Function to design and apply the Gammatone filter bank, rectification, and low-pass filtering
def process_signal_with_gammatone(input_signal, num_channels, lowcut, highcut, fs, overlap, distr, order, lpf_cutoff, lpf_order, ftype):
    """
    Applies the Gammatone filter bank, full-wave rectification, and low-pass filtering.

    Args:
        input_signal (numpy.ndarray): Input audio signal.
        num_channels (int): Number of channels for the Gammatone filter bank.
        lowcut (float): Low cutoff frequency for the Gammatone filter bank.
        highcut (float): High cutoff frequency for the Gammatone filter bank.
        fs (int): Sampling frequency.
        overlap (float): Overlap percentage for the Gammatone filter bank.
        distr (str): Distribution type for frequency bands ('erb', etc.).
        order (int): Order of the Gammatone filter bank.
        lpf_cutoff (float): Cutoff frequency for the low-pass filter.
        lpf_order (int): Order of the low-pass filter.
        ftype (str): Type of Gammatone filter ('fir' or 'iir').

    Returns:
        numpy.ndarray: The processed and combined signal.
    """
    # Design the Gammatone filter bank
    gammatone_bank = design_gammatone_fbank(num_channels, lowcut, highcut, distr, overlap, fs, ftype=ftype, order=order)
    
    # Apply the bandpass filter bank
    filtered_signals = apply_filter(input_signal, gammatone_bank)
    
    # Full-wave rectification
    rectified_signals = np.abs(np.array(filtered_signals))
    
    # Design the low-pass filter
    nyquist = fs / 2
    normalized_cutoff = lpf_cutoff / nyquist
    lpf_taps = ss.firwin(lpf_order + 1, normalized_cutoff)
    
    # Apply the low-pass filter to each channel
    lpf_filtered_signals = []
    for ch_signal in rectified_signals:
        lpf_filtered_signal = lfilter(lpf_taps, [1.0], ch_signal)
        lpf_filtered_signals.append(lpf_filtered_signal)
    
    # Combine all channels by summing them
    combined_signal = np.sum(lpf_filtered_signals, axis=0)
    
    # Normalize the combined signal
    output_signal = normalize_signal(combined_signal)
    
    return output_signal

# Function to perform a parameter sweep for the Gammatone filter bank
def parameter_sweep_gammatone(input_signal, fs, param_ranges, ftype):
    """
    Perform a parameter sweep for the Gammatone filter bank.

    Args:
        input_signal (numpy.ndarray): Input audio signal.
        fs (int): Sampling frequency.
        param_ranges (dict): Ranges for parameters to sweep over.
            Keys: 'num_channels', 'lowcut', 'highcut', 'overlap', 'order', 'lpf_cutoff', 'lpf_order'
        ftype (str): Type of Gammatone filter ('fir' or 'iir').
    """
    num_channels_range = param_ranges['num_channels']
    overlap_range = param_ranges['overlap']
    order_range = param_ranges['order']
    lpf_cutoff_range = param_ranges['lpf_cutoff']
    lpf_order_range = param_ranges['lpf_order']

    lowcut = param_ranges['lowcut']
    highcut = param_ranges['highcut']
    distr = param_ranges['distr']

    for num_channels in num_channels_range:
        for overlap in overlap_range:
            for order in order_range:
                for lpf_cutoff in lpf_cutoff_range:
                    for lpf_order in lpf_order_range:
                        print(f"Processing with num_channels={num_channels}, overlap={overlap}, order={order}, lpf_cutoff={lpf_cutoff}, lpf_order={lpf_order}, type={ftype}")
                        
                        # Process the signal
                        output_signal = process_signal_with_gammatone(
                            input_signal=input_signal,
                            num_channels=num_channels,
                            lowcut=lowcut,
                            highcut=highcut,
                            fs=fs,
                            overlap=overlap,
                            distr=distr,
                            order=order,
                            lpf_cutoff=lpf_cutoff,
                            lpf_order=lpf_order,
                            ftype=ftype
                        )

                        # Play the processed signal
                        display(Audio(output_signal, rate=fs))
                        
                        # Save the output signal
                        filename = f"gammatone_{ftype}_nc{num_channels}_ov{int(overlap*100)}_ord{order}_lpf{lpf_cutoff}_lpf_ord{lpf_order}.wav"
                        sf.write(filename, output_signal, fs)
                        print(f"Saved: {filename}")




In [ ]:
# Define parameter ranges for the sweep
param_ranges_gammatone = {
    'num_channels': [32],  # Testing typical values for cochlear implants
    'lowcut': 100,  # Fixed value from project requirements
    'highcut': 8000,  # Fixed value from project requirements
    'overlap': [0.1],  # Starting with 20% and 30% overlaps
    'order': [4],  # Filter orders to test
    'distr': 'erb',  # Fixed as ERB for this sweep
    'lpf_cutoff': [400],  # Testing 300 Hz, 400 Hz, and 500 Hz for LPF cutoff
    'lpf_order': [2]  # Testing LPF orders 32 and 64
}

# Call the parameter sweep function for Gammatone IIR
print("Sweeping Gammatone IIR:")
parameter_sweep_gammatone(input_signal=sound, fs=16000, param_ranges=param_ranges_gammatone, ftype='iir')

In [ ]:
# Define parameter ranges for the sweep
param_ranges_gammatone = {
    'num_channels': [32],  # Testing typical values for cochlear implants
    'lowcut': 100,  # Fixed value from project requirements
    'highcut': 8000,  # Fixed value from project requirements
    'overlap': [0.1],  # Starting with 20% and 30% overlaps
    'order': [4],  # Filter orders to test
    'distr': 'erb',  # Fixed as ERB for this sweep
    'lpf_cutoff': [400],  # Testing 300 Hz, 400 Hz, and 500 Hz for LPF cutoff
    'lpf_order': [2]  # Testing LPF orders 32 and 64
}

# Call the parameter sweep function for Gammatone FIR
print("Sweeping Gammatone FIR:")
parameter_sweep_gammatone(input_signal=sound, fs=16000, param_ranges=param_ranges_gammatone, ftype='fir')